# SPY Minute Data Downloader

This notebook downloads SPY minute-level data from Yahoo Finance and saves it to a CSV file in the specified format:
- date: Trading date (YYYY-MM-DD)
- minute: Minutes from market open (391 = 9:31 AM, 392 = 9:32 AM, etc.)
- open, low, high, close: OHLC prices
- volume: Trading volume

## 1. Import Required Libraries

Import necessary libraries including yfinance, pandas, and datetime for data manipulation and Yahoo Finance API access.

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, time
import os

print("Libraries imported successfully!")

## 2. Download SPY Data from Yahoo Finance

Use yfinance to download SPY minute-level data for a specified date range with proper interval settings.

In [ ]:
from datetime import datetime, timedelta

# Define the ticker symbol
ticker = "SPY"

days = 6
start_date = (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

# Download minute data from Yahoo Finance
print(f"Downloading {ticker} minute data from {start_date} to {end_date}...")
spy_data = yf.download(ticker, start=start_date, end=end_date, interval="1m")


In [ ]:

print(f"Downloaded {len(spy_data)} rows of minute data")
print(f"Date range: {spy_data.index[0]} to {spy_data.index[-1]}")
spy_data.head()


## 3. Process and Format the Data

Clean and format the downloaded data, ensuring proper column names and data types.

In [ ]:
# Reset index to make datetime a column
df = spy_data.reset_index()

# Drop the Adj Close column as it's not needed
# df = df.drop('adj_close', axis=1)

# Rename columns to match the desired format (lowercase)
df.columns = ['datetime', 'close', 'high', 'low', 'open', 'volume']

# datetime is utc, convert to pst
# df['datetime'] = df['datetime'].dt.tz_convert('America/Los_Angeles')

# Round the price columns to 2 decimal places
price_columns = ['open', 'high', 'low', 'close']
df[price_columns] = df[price_columns].round(2)

print(f"Processed {len(df)} rows")
print("Sample of processed data:")
df.head()

## 4. Add Minute Column Based on Time

Extract date and create a minute column that represents minutes from market open (e.g., 391 for 9:31 AM).

In [ ]:
# Extract date and time components
df['date'] = df['datetime'].dt.date
df['time'] = df['datetime'].dt.time

# Create minute column (minutes from market open)
# Market opens at 9:30 AM, so 9:31 AM = minute 391, 9:32 AM = minute 392, etc.
# Calculate minutes from midnight, then subtract market open minutes (9:30 AM = 570 minutes)
market_open_minutes = 9 * 60 + 30  # 9:30 AM in minutes from midnight (570)

df['minute'] = (df['datetime'].dt.hour * 60 + df['datetime'].dt.minute) - market_open_minutes

# Filter only regular trading hours (390-780, which is 9:30 AM to 4:00 PM)
df = df[(df['minute'] >= 390) & (df['minute'] <= 780)]

# Reorder columns to match desired format
df = df[['date', 'minute', 'open', 'low', 'high', 'close', 'volume']]

print(f"Filtered to {len(df)} rows of regular trading hours data")
print("Sample with minute column:")
df.head(10)

## 5. Write Data to CSV File

Export the formatted DataFrame to a CSV file with the specified column structure: date,minute,open,low,high,close,volume.

In [ ]:
# Create data directory if it doesn't exist
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Define output filename with date range
output_filename = f"{data_dir}/spy_minute.csv"

# Fix column name (volumn -> volume)
df = df.rename(columns={'volume': 'volumn'})

# Write to CSV file
df.to_csv(output_filename, index=False)

print(f"Data successfully written to: {output_filename}")
print(f"Total rows written: {len(df)}")
print(f"File size: {os.path.getsize(output_filename) / 1024:.2f} KB")

## 6. Verify the Output

Read back the CSV file and display sample rows to confirm the data format matches the required structure.

In [ ]:
# Read the CSV file back to verify format
verification_df = pd.read_csv(output_filename)

print("CSV file verification:")
print(f"Shape: {verification_df.shape}")
print(f"Columns: {list(verification_df.columns)}")
print("\nFirst 10 rows:")
print(verification_df.head(10))

print(f"\nSample data format matches requirement:")
print("date,minute,open,low,high,close,volumn")
for i in range(min(3, len(verification_df))):
    row = verification_df.iloc[i]
    print(f"{row['date']},{row['minute']},{row['open']},{row['low']},{row['high']},{row['close']},{row['volumn']}")

print(f"\nData range:")
print(f"Date range: {verification_df['date'].min()} to {verification_df['date'].max()}")
print(f"Minute range: {verification_df['minute'].min()} to {verification_df['minute'].max()}")